In [2]:
!pip install transformers openvino openvino-dev datasets
!pip install langchain

In [4]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
import torch

device = torch.device('cpu')
model.to(device)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [7]:
def generate_response(input_text):
    inputs = tokenizer.encode(input_text, return_tensors='pt').to(device)
    outputs = model.generate(inputs, max_length=15, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Example usage
user_input = "What is the capital of France?"
print(generate_response(user_input))


What is the capital of France?

The capital of France is Paris


In [8]:
!pip install python-libarchive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 10.7 MB/s eta 0:00:00


In [9]:
!pip install matplotlib-venn

In [10]:
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
import pydot

In [11]:
!pip install cartopy
import cartopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 32.5 MB/s eta 0:00:00


In [12]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# Load the health-related dataset from a local CSV file
dataset = load_dataset('csv', data_files='/content/intents (1).csv')

# Extract the data from the "train" split (which contains all the data)
full_dataset = dataset["train"]

# Split the dataset into train and test
train_dataset, test_dataset = train_test_split(full_dataset,test_size=0.2)

print(train_dataset)
print(test_dataset)

Generating train split: 0 examples [00:00, ? examples/s]

{'tag': ['animal bite', 'Cold', 'Eye Injury', 'Sun Burn', 'Fracture', 'Abrasions', 'Diarrhea', 'Abdonominal Pain', 'Skin problems', 'Insect Bites', 'CPR', 'Rash', 'Frost bite', 'snake bite', 'Drowning', 'stings', 'Sprains', 'Heat Stroke', 'Splinter', 'Chemical Burn', 'Headache', 'Normal Bleeding', 'Sore Throat', 'Fainting', 'Cough', 'Teeth', 'Fever', 'Choking', 'seizure', 'Cuts', 'Heat Exhaustion', 'Poison', 'Broken Toe', 'Rectal bleeding', 'Head Injury'], 'patterns__001': ['How do you treat a animal bite?', 'How do you treat a Cold?', 'How do you treat an eye Injury?', 'How do you treat Sun Burn?', 'How do you treat a Fracture?', 'how do you treat abrasions?', 'How do you treat Diarrhea?', 'How do you treat Abdonominal Pain?', 'How do you treat Skin problems?', 'How do you treat a Insect Bite?', 'How to give CPR??', 'How do you treat Rashes?', 'How do you treat a Frost bite?', 'How do you treat a snake bite?', 'What to do if someone is Drowning?', 'How do you treat Sting?', 'How do yo

In [1]:
!pip install accelerate>=0.21.0 -U  # Make sure accelerate is installed and up-to-date
!pip install --upgrade transformers  # Upgrade transformers to ensure compatibility

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# Define the maximum length for the model
MAX_LENGTH = 1024

# Load and preprocess the data
df = pd.read_csv('/content/intents (1).csv')
texts = df['tag'] + ': ' + df['responses__-']

# Save texts to a file
with open('train.txt', 'w', encoding='utf-8') as f:
    for text in texts:
        f.write(f"{text}\n")

# Load pre-trained model and tokenizer for causal language modeling
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Use AutoModelForCausalLM for language modeling instead of sequence classification
model = AutoModelForCausalLM.from_pretrained(model_name)

# Add padding token to tokenizer AND Model Config
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

# Tokenize the questions with truncation
def tokenize_and_truncate(question):
    return tokenizer.encode(question, return_tensors="np", max_length=MAX_LENGTH, truncation=True)

# Create dataset
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='train.txt',
    block_size=128
)

# Create data collator for causal language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # No masked language modeling
)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=250,
    per_device_train_batch_size=4,
    save_steps=10000,
    save_total_limit=2,
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained('./fine_tuned_gpt2_v3')
tokenizer.save_pretrained('./fine_tuned_gpt2_v3')

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (3919 > 1024). Running this sequence through the model will result in indexing errors


Step,Training Loss
500,0.480300
1000,0.021200
1500,0.015500
2000,0.014000


('./fine_tuned_gpt2_v3/tokenizer_config.json',
 './fine_tuned_gpt2_v3/special_tokens_map.json',
 './fine_tuned_gpt2_v3/vocab.json',
 './fine_tuned_gpt2_v3/merges.txt',
 './fine_tuned_gpt2_v3/added_tokens.json',
 './fine_tuned_gpt2_v3/tokenizer.json')

In [2]:
!pip install optimum[openvino]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.7/424.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.4/224.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 11.0 MB/s e

In [ ]:
from optimum.intel import OVModelForCausalLM
model_id = "gpt2"
model = OVModelForCausalLM.from_pretrained(model_id, export=True)


In [4]:
# Save model for faster loading later
model.save_pretrained("gpt2-ov")

# Initialize tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.save_pretrained("gpt2ov")

('gpt2ov/tokenizer_config.json',
 'gpt2ov/special_tokens_map.json',
 'gpt2ov/vocab.json',
 'gpt2ov/merges.txt',
 'gpt2ov/added_tokens.json',
 'gpt2ov/tokenizer.json')

In [5]:
# Load a saved model
model = OVModelForCausalLM.from_pretrained("gpt2-ov")
tokenizer = AutoTokenizer.from_pretrained("gpt2ov")

Compiling the model to CPU ...


In [6]:
from openvino.runtime import Core

# Initialize OpenVINO runtime
ie = Core()

# Load the model
model_xml = "/content/gpt2-ov/openvino_model.xml"  # Path to the XML file of the optimized model
model_bin = "/content/gpt2-ov/openvino_model.bin"  # Path to the BIN file of the optimized model
model = ie.read_model(model=model_xml, weights=model_bin)

# Compile the model for CPU
compiled_model = ie.compile_model(model=model, device_name="CPU")

# Create an inference request
infer_request = compiled_model.create_infer_request()

In [7]:
# Import necessary libraries
import csv
from openvino.runtime import Core
import numpy as np
import re
from difflib import get_close_matches


# Load and preprocess data
import csv

def load_data(file_path):
    intents = []
    with open(file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            # Print the first row to see the structure
            if len(intents) == 0:
                print("First row:", row)

            # Extract patterns and responses
            patterns = [row[key] for key in row.keys() if key.startswith('patterns__') and row[key]]
            responses = [row[key] for key in row.keys() if key.startswith('responses__') and row[key]]

            intents.append({
                'tag': row.get('tag', ''),  # Use .get() to avoid KeyError if 'tag' is missing
                'patterns': patterns,
                'responses': responses
            })
    return intents

# Try loading the data
try:
    intents = load_data('/content/intents (1).csv')
    print(f"Loaded {len(intents)} intents")
    # Print the first intent as an example
    if intents:
        print("First intent:", intents[0])
except Exception as e:
    print(f"An error occurred: {e}")
    # If there's an error, print out the first few lines of the CSV file
    with open('/content/intents (1).csv', 'r', encoding='utf-8') as file:
        print("First few lines of the CSV file:")
        for i, line in enumerate(file):
            if i < 5:  # Print first 5 lines
                print(line.strip())
            else:
                break

# Load pre-trained model (pseudocode, actual implementation depends on the chosen model)
# model = load_pretrained_model()

# Optimize model using OpenVINO
ie = Core()
model = ie.read_model(model="/content/gpt2-ov/openvino_model.xml")
compiled_model = ie.compile_model(model=model)

def preprocess(text):
    # Convert to lowercase and remove punctuation
    return re.sub(r'[^\w\s]', '', text.lower())
# Inference function
def get_response(intents, user_input):
    user_input = preprocess(user_input)

    # Find the best matching pattern
    best_match = None
    best_score = 0

    for intent in intents:
        for pattern in intent['patterns']:
            pattern = preprocess(pattern)
            matches = get_close_matches(user_input, [pattern], n=1, cutoff=0.6)
            if matches and len(matches[0]) / len(user_input) > best_score:
                best_match = intent
                best_score = len(matches[0]) / len(user_input)

    if best_match:
        return best_match['responses'][0]
    else:
        return "I'm sorry, I don't understand that question. Can you please rephrase it?"

def infer(input_text):
    return get_response(intents, input_text)

# Example usage
user_input = "How do I treat a cut?"
response = infer(user_input)
print(f"User: {user_input}")
print(f"Bot: {response}")

# Test with a few more inputs
test_inputs = [
    "What should I do for a sprained ankle?",
    "How to cure a headache?",
    "What's the treatment for a snake bite?",
    "How to perform CPR?",
    "how to cure fever"
]

for input_text in test_inputs:
    response = infer(input_text)
    print(f"\nUser: {input_text}")
    print(f"Bot: {response}")

First row: {'\ufeff"tag"': 'Cuts', 'patterns__001': 'What to do if Cuts?', 'patterns__002': 'How to cure Cuts?', 'patterns__003': 'Which medicine to apply for Cuts?', 'patterns__004': 'what to apply on cuts?', 'patterns__005': 'Cuts', 'patterns__006': '', 'patterns__007': '', 'responses__-': 'Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied.', 'context_set': ''}
Loaded 44 intents
First intent: {'tag': '', 'patterns': ['What to do if Cuts?', 'How to cure Cuts?', 'Which medicine to apply for Cuts?', 'what to apply on cuts?', 'Cuts'], 'responses': ['Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick heali

In [ ]:
from transformers import pipeline

# Load fine-tuned model
model_path = "/content/fine_tuned_gpt2_v3"  # Path to the fine-tuned model directory
nlp = pipeline("text-generation", model=model_path, tokenizer=model_path)  # Specify tokenizer path as well

# Simple chatbot loop
while True:
    user_input = input("You: ")
    response = nlp(user_input, truncation=True, max_length=50)
    print(f"Bot: {response[0]['generated_text']}")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


You: Cuts
Bot: Cuts: Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile band
